In [39]:
import requests
import pandas as pd
import xml.etree.ElementTree as et
import json

In [25]:
url = "https://ratings.food.gov.uk/business-types/xml"
headers = {
    "x-api-version": "2"
}

response = requests.get(url, headers=headers)

with open('feed.xml', 'wb') as file:
    file.write(response.content)

xtree = et.parse("feed.xml")
xroot = xtree.getroot()

df_cols = ["id", "type"]
rows=[]

for node in xroot: 
    b_id = node.find("BusinessTypeId").text if node is not None else None
    b_type = node.find("BusinessTypeName").text if node is not None else None
    
    rows.append({"id": b_id, "type": b_type})


In [40]:
def lat_lon_from_postcode(postcode):
    postcode = postcode.replace(" ", "").lower()
    url = "https://api.postcodes.io/postcodes/" + postcode
    response = requests.get(url)
    js_resp = response.json()
    lat = str(js_resp['result']['latitude'])
    lon = str(js_resp['result']['longitude'])
    lat_lon = [lat, lon]
    return lat_lon

In [41]:
def url_maker(postcode):
    url_base = "https://ratings.food.gov.uk/enhanced-search/en-GB/^/^/DISTANCE/1/^/"
    lat_lon = lat_lon_from_postcode(postcode)
    lat, lon = lat_lon[0], lat_lon[1]
    url = url_base + lon + "/" + lat + "/1/10000/json"
    return url

In [119]:
def ratings_downloader(postcode):
    url = url_maker(postcode)
    headers = {
        "x-api-version": "1"
        }
    response = requests.get(url, headers=headers)

    with open('feed.json', 'wb') as file:
        file.write(response.content)
        


In [120]:
def make_results_list(establishment):
    establishments_dict = {} 
    for k in ("BusinessName", "AddressLine1", "AddressLine2", "PostCode", "RatingValue"):
        establishments_dict[k] = establishment.get(k, None)
            
    establishments_dict['Latitude'] = establishment['Geocode']['Latitude']
    establishments_dict['Longitude'] = establishment['Geocode']['Longitude']

    if all(establishments_dict.values()):
        results.append(establishments_dict) 


In [124]:
def filter_results(postcode, rating_value):

    ratings_downloader(postcode)
    
    json_file_path = "feed.json"
    with open(json_file_path, 'r') as j:
        parsed = json.loads(j.read())
    for establishment in parsed["FHRSEstablishment"]["EstablishmentCollection"]["EstablishmentDetail"]:
        if rating_value:
            try:
                dog = int(establishment['RatingValue'])
            except:
                continue
            int_rv = int(establishment['RatingValue'])
            if rating_value[0] == "=":
                if int_rv == int(rating_value[1]):
                    make_results_list(establishment) 
            elif rating_value[0] == ">":
                if int_rv > int(rating_value[1]):
                     make_results_list(establishment) 
            elif rating_value[0] == "<":
                if int_rv < int(rating_value[1]):
                    make_results_list(establishment)
            else:
                continue
        else:
            make_results_list(establishment) 
    return results

In [139]:
results = []
import folium

def make_hygeine_map(postcode, rating_value = None):
    lat_lon = lat_lon_from_postcode(postcode) 
    center_lat, center_long = lat_lon[0], lat_lon[1]
    hyg_map = folium.Map([center_lat, center_long])
    limit=10000
    
    filter_results(postcode, rating_value)
    
    for business in results[:limit]:
        lat = business['Latitude']
        long = business['Longitude']
        name = business['BusinessName']
        hygeine = business['RatingValue']
        postcode = business['PostCode']
        details = "{} :\n {} hygiene:{}".format(name, postcode, hygeine)

        popup = folium.Popup(details, parse_html=True)
    
        marker = folium.Marker(location=[lat, long], popup=popup)
    
        marker.add_to(hyg_map)
    return hyg_map
    
make_hygeine_map('W1D 6BZ', rating_value = '>3')